In [2]:

import pandas as pd
import numpy as np

# Đọc dữ liệu Titanic
df = pd.read_csv('titanic.csv')

print("📊 DỮ LIỆU TITANIC")
print("="*60)
print(df.head(3))
print(f"\nKích thước: {df.shape[0]} hàng, {df.shape[1]} cột")
print(f"\nThông tin các cột:")
print(df.info())

📊 DỮ LIỆU TITANIC
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  

Kích thước: 891 hàng, 12 cột

Thông tin các cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    i

In [3]:
# ============================================
# 1. SERIES.APPLY() - ÁP DỤNG HÀM CHO TỪNG PHẦN TỬ TRONG 1 CỘT
# ============================================
# Mục đích: Trích xuất TITLE (danh xưng) từ cột Name
# VD: "Braund, Mr. Owen Harris" → "Mr"
#     "Cumings, Mrs. John Bradley..." → "Mrs"

def extract_title(name):
    # Tách theo dấu phẩy, lấy phần sau
    after_comma = name.split(',')[1]
    # Tách theo dấu chấm, lấy phần trước (title)
    title = after_comma.split('.')[0]
    # Xóa khoảng trắng
    return title.strip()

# Áp dụng hàm cho TỪNG phần tử trong cột Name
df['Title'] = df['Name'].apply(extract_title)

print("📝 VÍ DỤ 1: SERIES.APPLY() - Trích xuất Title từ Name")
print("="*60)
print(df[['Name', 'Title']].head(10))
print(f"\nCác Title xuất hiện: {df['Title'].value_counts().to_dict()}")

📝 VÍ DỤ 1: SERIES.APPLY() - Trích xuất Title từ Name
                                                Name   Title
0                            Braund, Mr. Owen Harris      Mr
1  Cumings, Mrs. John Bradley (Florence Briggs Th...     Mrs
2                             Heikkinen, Miss. Laina    Miss
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)     Mrs
4                           Allen, Mr. William Henry      Mr
5                                   Moran, Mr. James      Mr
6                            McCarthy, Mr. Timothy J      Mr
7                     Palsson, Master. Gosta Leonard  Master
8  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)     Mrs
9                Nasser, Mrs. Nicholas (Adele Achem)     Mrs

Các Title xuất hiện: {'Mr': 517, 'Miss': 182, 'Mrs': 125, 'Master': 40, 'Dr': 7, 'Rev': 6, 'Col': 2, 'Mlle': 2, 'Major': 2, 'Ms': 1, 'Mme': 1, 'Don': 1, 'Lady': 1, 'Sir': 1, 'Capt': 1, 'the Countess': 1, 'Jonkheer': 1}


In [4]:
# ============================================
# 2. DATAFRAME.APPLY(AXIS=1) - ÁP DỤNG HÀM CHO TỪNG HÀNG
# ============================================
# Mục đích: Tính FamilySize = SibSp + Parch + 1
# SibSp: số anh chị em/vợ chồng
# Parch: số cha mẹ/con cái
# +1: chính bản thân người đó

def compute_family_size(row):
    # row là 1 Series, truy cập như dictionary
    # row['SibSp'] = số anh chị em/vợ chồng
    # row['Parch'] = số cha mẹ/con cái
    return row['SibSp'] + row['Parch'] + 1

# Áp dụng hàm cho TỪNG HÀNG (axis=1)
df['FamilySize'] = df.apply(compute_family_size, axis=1)

print("\n👨‍👩‍👧‍👦 VÍ DỤ 2: DATAFRAME.APPLY(AXIS=1) - Tính FamilySize")
print("="*60)
print(df[['Name', 'SibSp', 'Parch', 'FamilySize']].head(10))
print(f"\nPhân bố FamilySize:")
print(df['FamilySize'].value_counts().sort_index())


👨‍👩‍👧‍👦 VÍ DỤ 2: DATAFRAME.APPLY(AXIS=1) - Tính FamilySize
                                                Name  SibSp  Parch  FamilySize
0                            Braund, Mr. Owen Harris      1      0           2
1  Cumings, Mrs. John Bradley (Florence Briggs Th...      1      0           2
2                             Heikkinen, Miss. Laina      0      0           1
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)      1      0           2
4                           Allen, Mr. William Henry      0      0           1
5                                   Moran, Mr. James      0      0           1
6                            McCarthy, Mr. Timothy J      0      0           1
7                     Palsson, Master. Gosta Leonard      3      1           5
8  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)      0      2           3
9                Nasser, Mrs. Nicholas (Adele Achem)      1      0           2

Phân bố FamilySize:
FamilySize
1     537
2     161
3     102
4      29

In [5]:
# ============================================
# 3. SERIES.MAP() - ÁNH XẠ GIÁ TRỊ ĐƠN GIẢN
# ============================================
# Mục đích: Chuyển Sex (male/female) thành số (1/0)
# male → 1
# female → 0

# Cách 1: Dùng dictionary
sex_mapping = {'male': 'nam', 'female': 'nu'}
df['SexCode'] = df['Sex'].map(sex_mapping)

# Cách 2: Dùng hàm (linh hoạt hơn)
def encode_sex(sex):
    """Chuyển Sex thành số"""
    if sex == 'male':
        return 'nam'
    else:
        return 'nu'

df['SexCode2'] = df['Sex'].map(encode_sex)

print("\n🚹🚺 VÍ DỤ 3: SERIES.MAP() - Mã hóa Sex thành số")
print("="*60)
print(df[['Name', 'Sex', 'SexCode', 'SexCode2']].head(10))


🚹🚺 VÍ DỤ 3: SERIES.MAP() - Mã hóa Sex thành số
                                                Name     Sex SexCode SexCode2
0                            Braund, Mr. Owen Harris    male     nam      nam
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female      nu       nu
2                             Heikkinen, Miss. Laina  female      nu       nu
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female      nu       nu
4                           Allen, Mr. William Henry    male     nam      nam
5                                   Moran, Mr. James    male     nam      nam
6                            McCarthy, Mr. Timothy J    male     nam      nam
7                     Palsson, Master. Gosta Leonard    male     nam      nam
8  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  female      nu       nu
9                Nasser, Mrs. Nicholas (Adele Achem)  female      nu       nu


In [6]:
# ============================================
# 4. DATAFRAME.APPLYMAP() - ÁP DỤNG CHO TỪNG Ô
# ============================================
# Mục đích: Thay thế NaN bằng giá trị mặc định
# (Chỉ để minh họa - thực tế dùng fillna() nhanh hơn!)

# Tạo DataFrame nhỏ để demo
mini_df = df[['Age', 'Fare', 'SibSp']].head(10).copy()

def replace_nan_with_zero(value):
    """
    Thay NaN bằng 0, giữ nguyên các giá trị khác
    """
    if pd.isna(value):  # Kiểm tra có phải NaN không
        return 0
    return value

# Áp dụng cho TỪNG Ô trong DataFrame
mini_df_filled = mini_df.applymap(replace_nan_with_zero)

print("\n🔢 VÍ DỤ 4: DATAFRAME.APPLYMAP() - Thay NaN bằng 0")
print("="*60)
print("Trước khi áp dụng:")
print(mini_df)
print("\nSau khi áp dụng:")
print(mini_df_filled)

# ⚠️ LƯU Ý: Trong pandas hiện đại, nên dùng .map() thay vì .applymap()
# hoặc tốt nhất là dùng fillna() cho nhanh!


🔢 VÍ DỤ 4: DATAFRAME.APPLYMAP() - Thay NaN bằng 0
Trước khi áp dụng:
    Age     Fare  SibSp
0  22.0   7.2500      1
1  38.0  71.2833      1
2  26.0   7.9250      0
3  35.0  53.1000      1
4  35.0   8.0500      0
5   NaN   8.4583      0
6  54.0  51.8625      0
7   2.0  21.0750      3
8  27.0  11.1333      0
9  14.0  30.0708      1

Sau khi áp dụng:
    Age     Fare  SibSp
0  22.0   7.2500      1
1  38.0  71.2833      1
2  26.0   7.9250      0
3  35.0  53.1000      1
4  35.0   8.0500      0
5   0.0   8.4583      0
6  54.0  51.8625      0
7   2.0  21.0750      3
8  27.0  11.1333      0
9  14.0  30.0708      1


C:\Users\Duy Toan\AppData\Local\Temp\ipykernel_21656\1311035382.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mini_df_filled = mini_df.applymap(replace_nan_with_zero)


In [10]:
# ============================================
# 5. GROUPBY().TRANSFORM() - TẠO FEATURE THEO NHÓM
# ============================================
# Mục đích: Chuẩn hóa Fare theo từng Pclass bằng Z-score
# Z-score = (giá trị - trung bình) / độ lệch chuẩn
# → Các giá trị cao hơn trung bình → z > 0
# → Các giá trị thấp hơn trung bình → z < 0

def zscore(series):
    """
    Tính Z-score cho 1 Series
    
    Z-score cho biết giá trị cách trung bình bao nhiêu độ lệch chuẩn
    """
    mean = series.mean()
    std = series.std()
    if std == 0:  # Tránh chia cho 0
        return series - mean
    return (series - mean) / std

# Chuẩn hóa Fare trong TỪNG Pclass
# transform() giữ nguyên kích thước DataFrame gốc
df['Fare_zscore_by_Pclass'] = df.groupby('Pclass')['Fare'].transform(zscore)

print("\n📊 VÍ DỤ 5: GROUPBY().TRANSFORM() - Chuẩn hóa Fare theo Pclass")
print("="*60)
print(df[['Name', 'Pclass', 'Fare', 'Fare_zscore_by_Pclass']].head(15))

# Giải thích kết quả
print("\n💡 Giải thích:")
print("- Z-score dương (>0): Fare cao hơn trung bình của lớp đó")
print("- Z-score âm (<0): Fare thấp hơn trung bình của lớp đó")
print("- Z-score gần 0: Fare gần trung bình")
print(f"\nTrung bình Fare theo Pclass:")
print(df.groupby('Pclass')['Fare'].mean())


📊 VÍ DỤ 5: GROUPBY().TRANSFORM() - Chuẩn hóa Fare theo Pclass
                                                 Name  Pclass     Fare  \
0                             Braund, Mr. Owen Harris       3   7.2500   
1   Cumings, Mrs. John Bradley (Florence Briggs Th...       1  71.2833   
2                              Heikkinen, Miss. Laina       3   7.9250   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)       1  53.1000   
4                            Allen, Mr. William Henry       3   8.0500   
5                                    Moran, Mr. James       3   8.4583   
6                             McCarthy, Mr. Timothy J       1  51.8625   
7                      Palsson, Master. Gosta Leonard       3  21.0750   
8   Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)       3  11.1333   
9                 Nasser, Mrs. Nicholas (Adele Achem)       2  30.0708   
10                    Sandstrom, Miss. Marguerite Rut       3  16.7000   
11                           Bonnell, Miss. Eliza

In [11]:
# ============================================
# 6. GROUPBY().AGG() - TỔNG HỢP NHANH
# ============================================
# Mục đích: Tính nhiều thống kê theo nhóm Pclass
# - Tỷ lệ sống sót (survival rate)
# - Tuổi trung bình (average age)
# - Số lượng hành khách

survival_stats = df.groupby('Pclass').agg({
    'Survived': ['mean', 'sum', 'count'],  # Tỷ lệ sống, tổng sống, tổng người
    'Age': ['mean', 'median'],              # Tuổi TB, tuổi giữa
    'Fare': ['mean', 'max', 'min']          # Giá TB, cao nhất, thấp nhất
})

print("\n📈 VÍ DỤ 6: GROUPBY().AGG() - Thống kê theo Pclass")
print("="*60)
print(survival_stats)

# Đặt tên cột rõ ràng hơn
survival_stats_clean = df.groupby('Pclass').agg(
    survival_rate=('Survived', 'mean'),
    total_survived=('Survived', 'sum'),
    passenger_count=('Survived', 'count'),
    avg_age=('Age', 'mean'),
    avg_fare=('Fare', 'mean')
).round(2)

print("\n📋 Bảng thống kê rõ ràng hơn:")
print(survival_stats_clean)

print("\n💡 Nhận xét:")
print("- Pclass 1 (hạng nhất): tỷ lệ sống sót cao nhất")
print("- Pclass 3 (hạng ba): tỷ lệ sống sót thấp nhất")
print("- Fare (giá vé) tỷ lệ thuận với Pclass")


📈 VÍ DỤ 6: GROUPBY().AGG() - Thống kê theo Pclass
        Survived                   Age              Fare               
            mean  sum count       mean median       mean       max  min
Pclass                                                                 
1       0.629630  136   216  38.233441   37.0  84.154687  512.3292  0.0
2       0.472826   87   184  29.877630   29.0  20.662183   73.5000  0.0
3       0.242363  119   491  25.140620   24.0  13.675550   69.5500  0.0

📋 Bảng thống kê rõ ràng hơn:
        survival_rate  total_survived  passenger_count  avg_age  avg_fare
Pclass                                                                   
1                0.63             136              216    38.23     84.15
2                0.47              87              184    29.88     20.66
3                0.24             119              491    25.14     13.68

💡 Nhận xét:
- Pclass 1 (hạng nhất): tỷ lệ sống sót cao nhất
- Pclass 3 (hạng ba): tỷ lệ sống sót thấp nhất
- Fare (g

In [ ]:
# ============================================
# 7. GROUPBY().APPLY() - LINH HOẠT NHẤT
# ============================================
# Mục đích: Lấy TOP 2 hành khách trả Fare cao nhất mỗi Pclass

def top_2_by_fare(group_df):
    """
    Hàm lấy 2 hàng có Fare cao nhất trong 1 nhóm
    
    Args:
        group_df: DataFrame của 1 nhóm (VD: tất cả hành khách Pclass=1)
        
    Returns:
        DataFrame chỉ có 2 hàng có Fare cao nhất
    """
    # Sắp xếp theo Fare giảm dần, lấy 2 hàng đầu
    return group_df.nlargest(2, 'Fare')

# Áp dụng hàm cho TỪNG NHÓM Pclass
top_passengers = df.groupby('Pclass').apply(top_2_by_fare)

print("\n🏆 VÍ DỤ 7: GROUPBY().APPLY() - Top 2 Fare cao nhất mỗi Pclass")
print("="*60)
print(top_passengers[['Name', 'Pclass', 'Fare', 'Survived']])

print("\n💡 Giải thích:")
print("- groupby('Pclass') chia thành 3 nhóm (Pclass 1, 2, 3)")
print("- apply(top_2_by_fare) chạy hàm trên TỪNG nhóm")
print("- Mỗi nhóm trả về 2 hàng → Tổng 6 hàng (3 nhóm × 2)")
print("- Hàm apply() linh hoạt nhất nhưng chậm nhất!")

In [ ]:
# ============================================
# 8. ROLLING().APPLY() - CỬA SỔ TRƯỢT
# ============================================
# Mục đích: Tính median Fare với cửa sổ 5 hàng liên tiếp
# Cửa sổ trượt: [hàng 1-5], [hàng 2-6], [hàng 3-7], ...

# Sắp xếp theo PassengerId để đảm bảo thứ tự
df_sorted = df.sort_values('PassengerId').copy()

# Tính median Fare trong cửa sổ 5 hàng
df_sorted['Fare_median_rolling5'] = df_sorted['Fare'].rolling(window=5).apply(
    lambda x: np.median(x), 
    raw=True  # raw=True nhanh hơn (truyền numpy array thay vì Series)
)

print("\n🪟 VÍ DỤ 8: ROLLING().APPLY() - Median Fare cửa sổ 5 hàng")
print("="*60)
print(df_sorted[['PassengerId', 'Fare', 'Fare_median_rolling5']].head(10))

print("\n💡 Giải thích:")
print("- 4 hàng đầu = NaN vì chưa đủ 5 hàng để tính")
print("- Hàng 5: median của [Fare hàng 1-5]")
print("- Hàng 6: median của [Fare hàng 2-6]")
print("- Cửa sổ 'lướt' qua dữ liệu → Rolling window")
print("- Dùng cho time series (dữ liệu theo thời gian)")

In [ ]:
# ============================================
# 9. PIPE() - XÂU CHUỖI PIPELINE
# ============================================
# Mục đích: Đóng gói nhiều bước xử lý thành pipeline rõ ràng

def clean_and_engineer(df):
    """
    Hàm làm sạch và tạo features
    Pipeline gồm nhiều bước
    """
    df = df.copy()  # Tránh sửa DataFrame gốc
    
    # Bước 1: Điền NaN cho Age bằng median
    df['Age'] = df['Age'].fillna(df['Age'].median())
    
    # Bước 2: Tạo feature IsChild (dưới 18 tuổi)
    df['IsChild'] = (df['Age'] < 18).astype(int)
    
    # Bước 3: Tạo feature FamilySize
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # Bước 4: Tạo feature IsAlone (đi một mình)
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    return df

# Sử dụng pipe() để áp dụng hàm
df_processed = (df
    .pipe(clean_and_engineer)  # Bước 1: Làm sạch và tạo features
    .assign(                    # Bước 2: Thêm features bằng assign()
        AgeGroup=lambda x: pd.cut(x['Age'], bins=[0, 12, 18, 60, 100], 
                                  labels=['Child', 'Teen', 'Adult', 'Senior'])
    )
)

print("\n🔗 VÍ DỤ 9: PIPE() - Pipeline xử lý dữ liệu")
print("="*60)
print("Features mới được tạo:")
print(df_processed[['Name', 'Age', 'IsChild', 'FamilySize', 'IsAlone', 'AgeGroup']].head(10))

print("\n💡 Ưu điểm của pipe():")
print("- Code rõ ràng, dễ đọc")
print("- Dễ test từng hàm riêng biệt")
print("- Dễ thêm/bớt bước trong pipeline")
print("- Tránh tạo nhiều biến trung gian")

In [ ]:
# ============================================
# 🎯 TÓM TẮT: KHI NÀO DÙNG HÀM NÀO?
# ============================================
print("\n" + "="*70)
print("📚 TÓM TẮT CÁC LOẠI HÀM TÙY CHỈNH TRONG PANDAS")
print("="*70)

summary = """
┌─────────────────────────┬──────────────────────────┬─────────────────────┐
│ HÀM                     │ KHI NÀO DÙNG             │ VÍ DỤ               │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 1. Series.apply()       │ Xử lý TỪNG PHẦN TỬ      │ Trích Title từ Name │
│                         │ của 1 cột                │                     │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 2. DataFrame.apply()    │ Xử lý TỪNG HÀNG          │ Tính FamilySize     │
│    (axis=1)             │ (cần nhiều cột)          │ từ SibSp + Parch    │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 3. Series.map()         │ ÁNH XẠ giá trị đơn giản  │ Sex → SexCode       │
│                         │ (dict hoặc hàm)          │ male→1, female→0    │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 4. DataFrame.applymap() │ Xử lý TỪNG Ô             │ Thay NaN bằng 0     │
│    (ít dùng)            │ (ưu tiên vectorized!)    │ (nên dùng fillna)   │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 5. groupby().transform()│ Tạo feature THEO NHÓM    │ Z-score Fare        │
│                         │ (giữ size gốc)           │ theo từng Pclass    │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 6. groupby().agg()      │ TỔNG HỢP nhanh theo nhóm │ Survival rate       │
│                         │ (mean, sum, count...)    │ theo Pclass         │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 7. groupby().apply()    │ Logic PHỨC TẠP theo nhóm │ Top 2 Fare          │
│                         │ (linh hoạt nhất)         │ mỗi Pclass          │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 8. rolling().apply()    │ CỬA SỔ TRƯỢT            │ Median Fare         │
│                         │ (time series)            │ 5 hàng liên tiếp    │
├─────────────────────────┼──────────────────────────┼─────────────────────┤
│ 9. pipe()               │ XÂU CHUỖI nhiều bước     │ Pipeline xử lý      │
│                         │ (clean → engineer)       │ dữ liệu gọn gàng    │
└─────────────────────────┴──────────────────────────┴─────────────────────┘

⚡ QUY TẮC VÀNG:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ ƯU TIÊN vectorized operations (nhanh hơn 100-1000 lần!)
   → df['A'] + df['B'], df['C'].str.upper(), np.where(), pd.cut()

⚠️  CHỈ DÙNG apply() khi:
   → Logic phức tạp KHÔNG có vectorized method
   → Cần xử lý NHIỀU CỘT cùng lúc (row-wise)
   → GroupBy với logic đặc biệt

❌ TRÁNH apply() khi có thể dùng:
   → Phép toán cột: df['A'] + df['B'] thay vì apply(lambda x: x['A']+x['B'])
   → String methods: .str.contains() thay vì apply(lambda x: 'word' in x)
   → Fillna, replace, map thay vì apply(lambda x: 0 if pd.isna(x) else x)

🏆 THỨ TỰ ƯU TIÊN (từ NHANH → CHẬM):
   1. Vectorized operations (nhanh nhất ⚡⚡⚡)
   2. map() với dictionary (nhanh ⚡⚡)
   3. transform() / agg() (trung bình ⚡)
   4. apply() trên Series (chậm 🐢)
   5. apply(axis=1) trên DataFrame (chậm hơn 🐢🐢)
   6. groupby().apply() (chậm nhất 🐢🐢🐢)
"""

print(summary)
print("="*70)
print("✓ Đã học xong các loại hàm tùy chỉnh trong pandas!")